In [1]:
from src.api import DataAPI , Predictor

DataAPI.update()
DataAPI.prepare_train_data()
Predictor.update_factors()

Update Files
Wed Jul  3 00:45:31 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.00 Secs
Wed Jul  3 00:45:31 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.03 Secs
Wed Jul  3 00:45:31 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.02 Secs
Wed Jul  3 00:45:33 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.28 Secs
Wed Jul  3 00:45:37 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.43 Secs


TypeError: reduce() of empty iterable with no initial value

In [1]:
from src.api import Predictor
Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gruRES_day] predicting!
--------------------------------------------------------------------------------


In [2]:
from src.api import ModelTestor
ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
